In [1]:
import pandas as pd
import re
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np

from scipy import stats
from scipy.stats import f_oneway, mannwhitneyu, kruskal
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

import sys
directory = '/content/drive/Shared drives/ICS 691G Research Project/Research Project/Data Analysis'
sys.path.append(directory)

import bug_fix_list
import internal_list
import external_list
import functional_list
import code_smell_list

bug_words = bug_fix_list.bug_words
internal_words = internal_list.internal_words
external_words = external_list.external_words
functional_words = functional_list.functional_words
smell_words = code_smell_list.smell_words


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# How do time-to-first-review and time-to-merge differ between PRs with vs. without SAR patterns?
# -- How do these metrics vary by agent?
# -- How do they compare against non-SAR PRs?
# Compare individual agent times for SAR vs non-SAR

print('Starting script.')
pd.set_option('display.max_rows', None)
pull_request_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")
comments_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_comments.parquet")
pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_task_type.parquet")
print('Finished querying parquets.')
print(len(pull_request_df))
pull_request_df.rename(columns={'id': 'pr_id'}, inplace=True)

Starting script.


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/688396b205d1d6f23ea2db87/a2ee09c5cdb6feaa352311ab9e29183c7db352bf6b02539c8b36b2f63fef62f2?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pull_request.parquet%3B+filename%3D%22pull_request.parquet%22%3B&Expires=1764812658&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0ODEyNjU4fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjg4Mzk2YjIwNWQxZDZmMjNlYTJkYjg3L2EyZWUwOWM1Y2RiNmZlYWEzNTIzMTFhYjllMjkxODNjN2RiMzUyYmY2YjAyNTM5YzhiMzZiMmY2M2ZlZjYyZjJcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=t3Ot17q9UhNsFr-KKdepa~eoNzo7HFwc0I1sWPLZuMPj2WVw4mExEhnmUf9YH~ZuZ4q3GtTfbI7riy7Db~VRKvu4sN9cAQf4cWCO6B9Xpg3xiiKxsllstM9fJ3yHGtHOmwsPIsOIU0LibBQkF0yWUkkaVyg1rbR8ouQHK~xXWAWLApw2LjYvU6mutcSRfkedomyec8xakIn3pyraiTsgABFOotMkUKGhr~zVZJReUG0y3OIC6b3iFGl4GfN4D9vpsctpO-69wx4TddgSQ74JW0I76CtOw~vlyD9UZlr43nLNNP49jgVhFfWctNQX7p

In [ ]:
# need body from pr_comments
combined_df = pull_request_df.merge(
    comments_df[['id', 'body']].add_prefix('comment_'),
    how='left',
    left_on='pr_id',
    right_on='comment_id',
    suffixes=('', '_task')
)
combined_df.drop(columns=['comment_id'])

# only look at refactor
combined_df = combined_df.merge(
    pr_task_type_df[['id', 'type']],
    how='left',
    left_on='pr_id',
    right_on='id'
)
combined_df.rename(columns={'type': 'pr_type'}, inplace=True)
combined_df = combined_df.loc[combined_df['pr_type'].str.contains('refactor', na=False)].copy()

print(combined_df['pr_id'].nunique())

print(f"Length of combined_df: {len(combined_df)}")

In [ ]:
print('Beginning regex searches.')

def to_regex_pattern(word):
    return re.escape(word).replace(r'\*', '.*')

bug_patterns = re.compile('|'.join(to_regex_pattern(w) for w in bug_words), re.IGNORECASE)
internal_patterns = re.compile('|'.join(to_regex_pattern(w) for w in internal_words), re.IGNORECASE)
external_patterns = re.compile('|'.join(to_regex_pattern(w) for w in external_words), re.IGNORECASE)
functional_patterns = re.compile('|'.join(to_regex_pattern(w) for w in functional_words), re.IGNORECASE)
smell_patterns = re.compile('|'.join(to_regex_pattern(w) for w in smell_words), re.IGNORECASE)

all_patterns = re.compile(
    '|'.join(to_regex_pattern(w) for w in (bug_words + internal_words + external_words + functional_words + smell_words)),
    re.IGNORECASE
)

print('Finished assembling regex patterns.')


In [ ]:

print('Searching for sar patterns.')
# search for any pattern
combined_df['is_sar'] = (
    combined_df['body'].str.contains(all_patterns, na=False) |
    combined_df['comment_body'].str.contains(all_patterns, na=False) |
    combined_df['title'].str.contains(all_patterns, na=False)
)
print('Finished searching any pattern.')

# search for specific category of pattern (bug, internal, external, functional, smell)
combined_df['bug'] = (
    combined_df['body'].str.contains(bug_patterns, na=False) |
    combined_df['comment_body'].str.contains(bug_patterns, na=False) |
    combined_df['title'].str.contains(bug_patterns, na=False)
)
print('Finished searching bug patterns.')
combined_df['internal'] = (
    combined_df['body'].str.contains(internal_patterns, na=False) |
    combined_df['comment_body'].str.contains(internal_patterns, na=False) |
    combined_df['title'].str.contains(internal_patterns, na=False)
)
print('Finished searching internal patterns.')
combined_df['external'] = (
    combined_df['body'].str.contains(external_patterns, na=False) |
    combined_df['comment_body'].str.contains(external_patterns, na=False) |
    combined_df['title'].str.contains(external_patterns, na=False)
)
print('Finished searching external patterns.')
combined_df['functional'] = (
    combined_df['body'].str.contains(functional_patterns, na=False) |
    combined_df['comment_body'].str.contains(functional_patterns, na=False) |
    combined_df['title'].str.contains(functional_patterns, na=False)
)
print('Finished searching functional patterns.')
combined_df['smell'] = (
    combined_df['body'].str.contains(smell_patterns, na=False) |
    combined_df['comment_body'].str.contains(smell_patterns, na=False) |
    combined_df['title'].str.contains(smell_patterns, na=False)
)
print('Finished searching smell patterns.')

# search any category pattern in specific locations in the PR
combined_df['sar_in_pr_title'] = combined_df['title'].str.contains(all_patterns, na=False)
print('Finished searching for sar in title.')
combined_df['sar_in_pr_body'] = combined_df['body'].str.contains(all_patterns, na=False)
print('Finished searching for sar in body.')
combined_df['sar_in_pr_comment'] = combined_df['comment_body'].str.contains(all_patterns, na=False)
print('Finished searching for sar in comment.')

# add column for merge time in DAYS instead of seconds
combined_df['merge_time'] = (pd.to_datetime(combined_df['merged_at']) - pd.to_datetime(combined_df['created_at'])).dt.total_seconds() / 86400

In [ ]:

unique_prs = combined_df.drop_duplicates(subset=['pr_id'])

total_requests = (
    unique_prs
    .groupby(['agent', 'is_sar'])
    .size()
    # ** issue with my python LSP, not a problem at runtime **
    .reset_index(name='total_requests') # type: ignore
)

agents = unique_prs['agent'].unique()
sar_flags = [True, False]
all_groups = pd.MultiIndex.from_product([agents, sar_flags], names=['agent', 'is_sar'])

total_merged = (
    unique_prs[unique_prs['merged_at'].notna()]
    .groupby(['agent', 'is_sar'])['merge_time']
    .size()
    .reindex(all_groups, fill_value=0)
    .reset_index(name='total_merged') # type: ignore
)

# find average of both sar and non_sar, and drop unmerged PRs
average_merged = (
    unique_prs[unique_prs['merged_at'].notna()]
    .groupby(['agent', 'is_sar'])['merge_time']
    .mean()
    .round(2)
    .reindex(all_groups, fill_value='n/a')
    .reset_index(name='average_merge_time(days)') # type: ignore
)

sar_categories = (
    unique_prs
    .groupby(['agent', 'is_sar'])
    [['bug', 'internal', 'external', 'functional', 'smell']]
    .sum()
    .reset_index()
)

sar_locations = (
    unique_prs
    .groupby(['agent', 'is_sar'])
    [['sar_in_pr_title', 'sar_in_pr_body', 'sar_in_pr_comment']]
    .sum()
    .reset_index()
)


summary = (
    total_requests
    .merge(total_merged, on=['agent', 'is_sar'], how='left')
    .merge(average_merged, on=['agent', 'is_sar'], how='left')
    .merge(sar_categories, on=['agent', 'is_sar'], how='left')
    .merge(sar_locations, on=['agent', 'is_sar'], how='left')
)
summary['merge_rate(%)'] = ((summary['total_merged'] / summary['total_requests']).round(2) * 100).astype(int)

In [ ]:
combined_df

In [ ]:
summary

# Testing


## Two-Sample test of SAR and non-SAR PRs

In [ ]:
rq_testing = unique_prs[unique_prs['merged_at'].notna()][['agent', 'is_sar', 'merge_time']].copy()

# Check normality assumption to see if can use ANOVA test
# Shapiro-Wilk test
for agent in rq_testing['agent'].unique():
    for sar in [True, False]:
        data = rq_testing[(rq_testing['agent'] == agent) &
                          (rq_testing['is_sar'] == sar)]['merge_time']
        if len(data) >= 3:  # Need at least 3 samples
            stat, p = stats.shapiro(data)
            print(f"  {agent} (SAR={sar}): W={stat:.4f}, p={p:.4f}")

# Levene's test
print("\nLevene's test for homogeneity of variance:")
groups = [group['merge_time'].values for name, group in
          rq_testing.groupby(['agent', 'is_sar'])]
stat, p = stats.levene(*groups)
print(f"  Statistic={stat:.4f}, p={p:.4f}")

# Normality Assumptions:
- All Shapiro-Wilk p-values are less than 0.05 and Levene's p is less than 0.05, so normality and homogeneity assumptions are violated!!!
Use Kruskal-Wallis test instead

In [ ]:
rq_testing['agent_sar'] = rq_testing['agent'] + '_' + rq_testing['is_sar'].astype(str)
groups_kw = [group['merge_time'].values for name, group in
             rq_testing.groupby('agent_sar')]
stat, p = kruskal(*groups_kw)
print(f"Kruskal-Wallis H-statistic: {stat:.4f}, p-value: {p:.4f}")

Kruskal-Wallis H: 558.7417 and p < 0.05 means there is significant differences across the agent-SAR combos

In [ ]:
agent_results = []

for agent in rq_testing['agent'].unique():
    print(f"\n{agent}")
    print("-" * 60)

    agent_data = rq_testing[rq_testing['agent'] == agent]
    sar_data = agent_data[agent_data['is_sar'] == True]['merge_time']
    non_sar_data = agent_data[agent_data['is_sar'] == False]['merge_time']

    n_sar = len(sar_data)
    n_non_sar = len(non_sar_data)

    print(f"  SAR PRs: n={n_sar}, mean={sar_data.mean():.4f}, median={sar_data.median():.4f}")
    print(f"  Non-SAR PRs: n={n_non_sar}, mean={non_sar_data.mean():.4f}, median={non_sar_data.median():.4f}")

    if n_sar >= 3 and n_non_sar >= 3:
        # Use Mann-Whitney U (non-parametric)
        stat, p_val = mannwhitneyu(sar_data, non_sar_data, alternative='two-sided')
        test_used = "Mann-Whitney U test"

        # Calculate effect size (Cohen's d)
        pooled_std = np.sqrt(((n_sar-1)*sar_data.std()**2 +
                              (n_non_sar-1)*non_sar_data.std()**2) /
                             (n_sar + n_non_sar - 2))
        cohens_d = (sar_data.mean() - non_sar_data.mean()) / pooled_std

        print(f"  statistic={stat:.4f}, p-value={p_val:.4f}")
        print(f"  Cohen's d (effect size): {cohens_d:.4f}")

        agent_results.append({
            'agent': agent,
            'n_sar': n_sar,
            'n_non_sar': n_non_sar,
            'mean_sar': sar_data.mean(),
            'mean_non_sar': non_sar_data.mean(),
            'median_sar': sar_data.median(),
            'median_non_sar': non_sar_data.median(),
            'test': test_used,
            'p_value': p_val,
            'cohens_d': cohens_d,
            'significant': p_val < 0.05
        })
    else:
        print(f"  Insufficient data for statistical testing (need n>=3 for both groups)")

results_df = pd.DataFrame(agent_results)
if len(results_df) > 0:
    print(results_df.to_string(index=False))

- Only OpenAI-Codex had significant results, as the p-value was below 0.05
  - SAR PRs merge faster than non-SAR PRs
  - The mean for a SAR is 0.24 days while the mean for a non-SAR was 0.45 days
  - Median for SAR: 0.002 days (which is about 3 minutes), median of non-SAR: 0.001 days (about 1.4 minutes)
  - SAR patterns have slightly faster merges
  - Cohen's D = 0.07 which means it is a very small effect though

- Other agents did not have significant results (P > 0.05), but Copilot's SAR PRs tended to be slower. Claude also couldn't be tested as there were no merged SAR PRs in the dataset. All PRs that had SAR patterns were not merged, so no merge time

# Threats to Validity
- Failed normality: The medians are much lower, meaning data is right-skewed with outliers
- Very large sample size imbalance for all agents
- All Cohen's d values are very small, suggesting that any differences are not very meaningful :(


# Results
- Time-to-merge does not differ meaningfully between PRs with and without SAR patterns overall (p = 1.000).
- Significant variation exists across agents (p < 0.001), but the agent-SAR interaction is not significant (p = 0.221).
- Only OpenAI_Codex shows a statistically significant difference (p = 0.006), where SAR PRs merge slightly faster than non-SAR PRs (median: 0.002 vs 0.001 days). However, this difference is not practically meaningful (Cohen's d = -0.07).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Box plot by agent and SAR
ax1 = axes[0, 0]
rq_testing.boxplot(column='merge_time', by=['agent', 'is_sar'], ax=ax1)
ax1.set_title('Merge Time by Agent and SAR Pattern')
ax1.set_xlabel('Agent and SAR Pattern')
ax1.set_ylabel('Merge Time (days)')
plt.sca(ax1)
plt.xticks(rotation=45, ha='right')

# 2. Violin plot
ax2 = axes[0, 1]
sns.violinplot(data=rq_testing, x='agent', y='merge_time', hue='is_sar',
               split=True, ax=ax2)
ax2.set_title('Distribution of Merge Times')
ax2.set_ylabel('Merge Time (days)')
ax2.legend(title='Has SAR Pattern')

# 3. Mean comparison with error bars
ax3 = axes[1, 0]
means = rq_testing.groupby(['agent', 'is_sar'])['merge_time'].mean().unstack()
errors = rq_testing.groupby(['agent', 'is_sar'])['merge_time'].sem().unstack()
means.plot(kind='bar', yerr=errors, ax=ax3, capsize=4)
ax3.set_title('Mean Merge Time with Standard Error')
ax3.set_ylabel('Merge Time (days)')
ax3.set_xlabel('Agent')
ax3.legend(title='Has SAR Pattern', labels=['No SAR', 'Has SAR'])
plt.sca(ax3)
plt.xticks(rotation=45, ha='right')

# 4. Count of PRs
ax4 = axes[1, 1]
counts = pd.crosstab(rq_testing['agent'], rq_testing['is_sar'])
counts.plot(kind='bar', stacked=True, ax=ax4)
ax4.set_title('Number of PRs by Agent and SAR Pattern')
ax4.set_ylabel('Count')
ax4.set_xlabel('Agent')
ax4.legend(title='Has SAR Pattern', labels=['No SAR', 'Has SAR'])
plt.sca(ax4)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()